In [2]:
import numpy as np
import pandas as pd
import os
import csv
import re
from dotenv import load_dotenv
import json
from openai import OpenAI
from urllib.parse import urlparse
import hashlib
import pdfkit
import requests
from playwright.sync_api import sync_playwright
from openai import OpenAI
import json

In [3]:
path = '/home/cptaswadu/RESCUE-n8n/insurance'
load_dotenv(dotenv_path=os.path.join(path, ".env"))
openai_api_key = os.getenv("OPEN_AI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [10]:
cases = [
    {
        "id": "Case1",
        "patient_info": "An 8-year-old boy with neurodevelopmental delay and seizures. A prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by the genetic counselor to investigate potential underlying genetic causes that may guide diagnosis and future treatment decisions. There is also a family history of neurodevelopmental disorders, as his older brother was diagnosed with autism spectrum disorder. The patient is covered by UnitedHealthcare Choice Plus through a family plan and resides in New Jersey."
    },
    {
        "id": "Case2",
        "patient_info": "An 8-year-old boy with mild learning difficulties and no significant neurological symptoms. There is no family history of genetic conditions, and his prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by his primary care provider (PCP) to explore potential genetic factors as part of general health screening and educational planning. The patient is covered by UnitedHealthcare Choice Plus through a family plan and resides in New Jersey."
    },
    {
        "id": "Case3",
        "patient_info": "An 8-year-old boy with mild learning difficulties and subtle developmental concerns. While there are no significant neurological symptoms and no family history of genetic conditions, his prior chromosomal microarray test was negative. However, his primary care provider (PCP) noted mild motor delays and expressive language difficulties during recent evaluations, and recommended Whole Exome Sequencing (WES) to investigate possible underlying genetic factors that could inform further medical and developmental management. The patient is covered by UnitedHealthcare Choice Plus through a family plan and resides in New Jersey."
    }
]


In [ ]:
cases2 = [
    {
        "id": "Case4",
        "patient_info": "A 35-year-old woman with a strong family history of breast and ovarian cancer. Her mother was diagnosed with breast cancer at age 42, and her maternal aunt had ovarian cancer in her 50s. The patient herself has no history of cancer but has dense breast tissue and is considered at increased risk. The genetic counselor has recommended BRCA1/BRCA2 testing to assess her hereditary cancer risk and guide risk-reducing management decisions, including potential prophylactic options. The patient is covered by Aetna Open Access Managed Choice Plan and resides in California."
    },
    {
        "id": "Case5",
        "patient_info": "A 28-year-old woman with no family history of breast or ovarian cancer. The patient requested BRCA1/BRCA2 genetic testing after reading about genetic risks online. There were no prior specialist consultations or referrals, and no other clinical risk factors have been identified. The test was ordered directly by her primary care physician at the patient's request. The patient is enrolled in Aetna Open Access Managed Choice Plan and lives in Texas."
    },
    {
        "id": "Case6",
        "patient_info": "A 31-year-old woman with no known family history of breast or ovarian cancer. She has a history of multiple benign breast biopsies and dense breast tissue noted on mammograms. During her annual gynecologic exam, her OB-GYN expressed concern about her cumulative risk and recommended BRCA1/BRCA2 testing as part of a broader risk assessment strategy. While formal genetic counseling was not provided, the OB-GYN discussed the implications and potential management options if a mutation were identified. The patient is covered under Aetna Open Access Managed Choice Plan and resides in Illinois."
    }
]


In [23]:
case_ex = [
    {
        "id": "Case1",
        "patient_info": "An 8-year-old boy with neurodevelopmental delay and seizures. A prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by the genetic counselor to investigate potential underlying genetic causes that may guide diagnosis and future treatment decisions. There is also a family history of neurodevelopmental disorders, as his older brother was diagnosed with autism spectrum disorder. The patient is covered by United Healthcare Choice Plus through a family plan and resides in New Jersey."
    },
    {
        "id": "Case2",
        "patient_info": "An 8-year-old boy with mild learning difficulties and no significant neurological symptoms. There is no family history of genetic conditions, and his prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by his primary care provider (PCP) to explore potential genetic factors as part of general health screening and educational planning. The patient is covered by United Healthcare Choice Plus through a family plan and resides in New Jersey."
    },
    {
        "id": "Case3",
        "patient_info": "A 35-year-old woman with a strong family history of breast and ovarian cancer. Her mother was diagnosed with breast cancer at age 42, and her maternal aunt had ovarian cancer in her 50s. The patient herself has no history of cancer but has dense breast tissue and is considered at increased risk. The genetic counselor has recommended BRCA1/BRCA2 testing to assess her hereditary cancer risk and guide risk-reducing management decisions, including potential prophylactic options. The patient is covered by Aetna Open Access Managed Choice Plan and resides in California."
    },
    {
        "id": "Case4",
        "patient_info": "A 28-year-old woman with no family history of breast or ovarian cancer. The patient requested BRCA1/BRCA2 genetic testing after reading about genetic risks online. There were no prior specialist consultations or referrals, and no other clinical risk factors have been identified. The test was ordered directly by her primary care physician at the patient's request. The patient is enrolled in Aetna Open Access Managed Choice Plan and lives in Texas."
    }
]

In [30]:
def generate_search_query(patient_info):
    return f"{patient_info['Insurance Provider']} {patient_info['Plan']} {patient_info['Location']} {patient_info['Test']} coverage policy"

In [7]:
questions_file_path = "/home/cptaswadu/RESCUE-n8n/insurance/dataset/Insurance_Genetic_Testing_QA.json"

with open(questions_file_path, "r") as f:
    questions_data = json.load(f)

questions_list = questions_data["questions"]
print(questions_list)

[{'id': 'Q0', 'question': 'What is the requested genetic test? Please specify the name, gene content if known.', 'options': ['Free text answer']}, {'id': 'Q1', 'question': "Does the patient's age meet the policy's coverage criteria for this genetic test?", 'options': ['Yes', 'No', 'Not specified']}, {'id': 'Q2', 'question': 'Was the test ordered by a specialist or provider type approved by the policy?', 'options': ['Yes', 'No', 'Not specified']}, {'id': 'Q3', 'question': 'Does the patient’s clinical presentation meet the policy’s definition of medical necessity for this genetic test?', 'options': ['Yes', 'No']}, {'id': 'Q4', 'question': "According to the policy, does the patient's clinical presentation meet any recognized clinical guidelines that support the use of this genetic test?", 'options': ['Yes', 'No'], 'additional_if_yes': ['ACMG', 'NCCN', 'ASCO', 'USPSTF', 'ISPD', 'None of the above']}, {'id': 'Q5', 'question': 'Has the patient completed prerequisite evaluations required by t

In [27]:
prompt_template = """
You are a clinical insurance assistant specializing in genetic testing coverage policies.

Your task is to assist with genetic test coverage assessment using both the provided patient information and the most relevant and up-to-date insurance policy. If needed, use web search to find the applicable policy.

Instructions:
1. Search for the most relevant official insurance policy document based on the patient's insurance plan and requested genetic test.
2. Carefully analyze the policy in combination with the patient’s case information.
3. Answer the following standardized questions strictly based on the policy.
4. Provide your answers in JSON format only. Do not include any explanation or additional text.
5. Include the source URL of the policy document in your JSON output at the end.

PATIENT INFORMATION:
\"\"\"
{patient_info}
\"\"\"

QUESTIONS:
{questions}

Output your answers in JSON format only and please provide the source URL of the policy document.
"""


In [28]:
def run_llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a clinical insurance assistant specializing in genetic testing policy search and analysis."},
            {"role": "user", "content": prompt}
        ],
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "web_search",
                    "description": "Search the web for relevant insurance policy documents and guidelines."
                }
            }
        ],
        temperature=0
    )
    if response.choices[0].finish_reason == "tool_calls":
        tool_call = response.choices[0].message.tool_calls[0]

        tool_result = "Simulated search result: [Aetna policy found about genetic testing criteria...]"

        followup = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a clinical insurance assistant specializing in genetic testing policy search and analysis."},
                {"role": "user", "content": prompt},
                {"role": "assistant", "tool_calls": [tool_call]},
                {"role": "tool", "tool_call_id": tool_call.id, "content": tool_result},
            ],
            temperature=0
        )

        result_content = followup.choices[0].message.content

    else:
        result_content = response.choices[0].message.content

    return result_content


In [29]:
results = []

for case in case_ex:
    print(f"\n==== Running {case['id']} ====")

    question_texts = []
    for q in questions_list:
        question_text = f"{q['id']}. {q['question']} Options: {', '.join(q['options'])}"

        additional_conditions = {
            "additional_options_if_yes": "If Yes → choose:",
            "additional_options_if_no": "If No → choose:",
            "additional_input_if_yes": "If Yes → provide:"
        }

        for key, instruction in additional_conditions.items():
            if key in q and q[key]:
                value = q[key]
                if isinstance(value, list):
                    question_text += f" ({instruction} {', '.join(value)})"
                else:
                    question_text += f" ({instruction} {value})"

        question_texts.append(question_text)

    questions_text = "\n".join(question_texts)

    prompt = prompt_template.format(
        patient_info=case["patient_info"],
        questions=questions_text
    )

    llm_output = run_llm(prompt)

    result = {
        "case_id": case["id"],
        "llm_output": llm_output
    }
    results.append(result)

    print("\n[LLM Output]")
    print(llm_output)



==== Running Case1 ====

[LLM Output]
```json
{
  "Q0": "Whole exome sequencing (WES)",
  "Q1": "Yes",
  "Q2": "Yes",
  "Q3": "Yes",
  "Q4": "Yes",
  "Q5": "Yes",
  "Q6": "No",
  "Q7": "Yes",
  "Q8": "No",
  "Q9": "Yes",
  "Q10": "No",
  "Q11": "Yes",
  "Q12": "No",
  "Q13": "Yes",
  "Q14": "Yes",
  "Q15": "No",
  "Q16": "Yes",
  "Q17": "Submit a prior authorization request with supporting documentation including clinical notes, family history, and previous genetic testing results."
}
```

==== Running Case2 ====

[LLM Output]
```json
{
  "Q0": "Whole exome sequencing (WES)",
  "Q1": "Yes",
  "Q2": "No",
  "Q3": "No",
  "Q4": "No",
  "Q5": "Yes",
  "Q6": "No",
  "Q7": "No",
  "Q8": "Yes",
  "Q9": "No",
  "Q10": "No",
  "Q11": "Not specified",
  "Q12": "No",
  "Q13": "Yes",
  "Q14": "Not listed",
  "Q15": "No",
  "Q16": "No",
  "Q17": "The policy requires prior authorization, submission of medical necessity documentation, and possibly genetic counseling documentation if specified."
}
`

In [8]:
results = []

def format_questions(questions_list):
    question_texts = []
    
    for q in questions_list:
        question_text = f"{q['id']}. {q['question']} Options: {', '.join(q['options'])}"

        for key, value in q.items():
            if key.startswith("additional_if_") and value:
                instruction = "If Yes →" if "if_yes" in key else "If No →"
                additional_text = f"{instruction} {' / '.join(value)}"
                question_text += f" ({additional_text})"

        question_texts.append(question_text)

    return "\n".join(question_texts)



def run_case(case):
    """Run LLM for a single case and store result."""
    print(f"\n==== Running {case['id']} ====")

    questions_text = format_questions(questions_list)

    prompt = prompt_template.format(
        patient_info=case["patient_info"],
        questions=questions_text
    )

    llm_output = run_llm(prompt)

    result = {
        "case_id": case["id"],
        "llm_output": llm_output
    }

    print("\n[LLM Output]")
    print(llm_output)

    return result

In [11]:
for case in cases:
    result = run_case(case)
    results.append(result)


==== Running Case1 ====

[LLM Output]
```json
{
  "Q0": "Whole exome sequencing (WES)",
  "Q1": "Yes",
  "Q2": "Yes",
  "Q3": "Yes",
  "Q4": "Yes (ACMG)",
  "Q5": "Yes",
  "Q6": "No",
  "Q7": "Yes",
  "Q8": "No",
  "Q9": "Yes",
  "Q10": "No (Diagnostic)",
  "Q11": "Yes",
  "Q12": "Yes",
  "Q13": "Yes",
  "Q14": "Yes (81415, 81416)",
  "Q15": "No",
  "Q16": "Yes",
  "Q17": "Submit prior authorization request with clinical notes, genetic counseling documentation, and previous genetic test results.",
  "source_url": "https://www.uhcprovider.com/content/dam/provider/docs/public/policies/comm-medical-drug/whole-exome-sequencing.pdf"
}
```

==== Running Case2 ====

[LLM Output]
```json
{
  "Q0": "Whole exome sequencing (WES)",
  "Q1": "Yes",
  "Q2": "No",
  "Q3": "No",
  "Q4": "No",
  "Q5": "Yes",
  "Q6": "No",
  "Q7": "No",
  "Q8": "Yes",
  "Q9": "No",
  "Q10": "No",
  "Q11": "Not specified",
  "Q12": "No",
  "Q13": "Not specified",
  "Q14": "Not listed",
  "Q15": "No",
  "Q16": "No",
  "Q

In [9]:
for case in cases2:
    result = run_case(case)
    results.append(result)


==== Running Case3 ====

[LLM Output]
```json
{
  "Q0": "BRCA1/BRCA2 genetic testing",
  "Q1": "Yes",
  "Q2": "Yes",
  "Q3": "Yes",
  "Q4": "Yes (NCCN)",
  "Q5": "Not required",
  "Q6": "No",
  "Q7": "Yes",
  "Q8": "No",
  "Q9": "Yes",
  "Q10": "No (Risk Assessment)",
  "Q11": "Yes",
  "Q12": "No",
  "Q13": "Yes",
  "Q14": "Yes (CPT code 81211)",
  "Q15": "No",
  "Q16": "Yes",
  "Q17": "Submit a claim with the appropriate CPT code, include documentation of medical necessity, family history, and genetic counseling notes."
}
```

==== Running Case4 ====

[LLM Output]
```json
{
  "Q0": "BRCA1/BRCA2 genetic testing",
  "Q1": "Not specified",
  "Q2": "No",
  "Q3": "No",
  "Q4": "No",
  "Q5": "Not required",
  "Q6": "No",
  "Q7": "No",
  "Q8": "No",
  "Q9": "Yes",
  "Q10": "Yes",
  "Q11": "Yes",
  "Q12": "No",
  "Q13": "Yes",
  "Q14": "Yes",
  "Q15": "No",
  "Q16": "No",
  "Q17": "The policy requires a specialist referral, genetic counseling, and prior authorization. The claim must include 